# Introduction to NLP Fundaments in TensorFlow

NLP has the goal of driving information out of natural language(could be sequences,text or speech)

Another common term for NLP problems is sequence to sequence problems(seq2seq).

In [1]:
## Check for GPU
import tensorflow as tf
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Get helper functions

#import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys


## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text sample of tweets labelled as disaster or not disaster).

See the original source in kaggle.


In [4]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [5]:
#unzip the data
unzip_data('nlp_getting_started.zip')

## Visualizing the text dataset

To visualize our text samples, we first have to read them in.

We can use pandas for this.

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [8]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) #shuffle the complete data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
#what does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class are there?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [11]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Let's visualize some random trainig examples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher than total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index + 5].itertuples():
    _,text,target = row
    print(f"Target : {target}","(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("----\n")

Target : 1 (real disaster)
Text:
iembot_hfo : At 10:00 AM 2 NNW Hana [Maui Co HI] COUNTY OFFICIAL reports COASTAL FLOOD #Û_ http://t.co/Gg0dZSvBZ7) http://t.co/kBe91aRCdw

----

Target : 0 (not real disaster)
Text:
@AnnmarieRonan @niamhosullivanx I can't watch tat show its like a horror movie to me I get flashbacks an everything #traumatised

----

Target : 0 (not real disaster)
Text:
Brooklyn locksmith: domesticate emergency mechanic services circa the clock movement!: gba http://t.co/1Q6ccFfzV6

----

Target : 1 (real disaster)
Text:
A look at state actions a year after Ferguson's upheaval http://t.co/vXUFtVT9AU

----

Target : 0 (not real disaster)
Text:
First responders would never be sent to the wrong address again w/ #SmartTek's GPS-based panic button #mPERS #safety http://t.co/3OionqlFQL

----



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
# Use train_test_split to split data into train and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                        train_df_shuffled["target"].to_numpy(),
                                                                        test_size=0.1,
                                                                                random_state=42)

In [15]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [16]:
# check the shapes
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [17]:
# check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text to numbers

**Tokenization vs Embedding:** A straight mapping of character to number/ word to number. It can be a one-hot encoding or a label encoding. This data can be fed into the models. But, as the size of the text increases the size of the token matrix also increases. Where as embedding is a matrix which represent the relationship between words in a text. It comes in the form of a feature vector. The length of the embeddings are variable. The word embeddings are float values between 0 and 1, they can act as weights to the model layers; so that they can be updated dynamically during training. The length of word embeddings are chosen as multiples of 8. most common embedding sizes are 32,64,128,256,512. You can either use a custom embedding layer from keras or use a pretrained one.


When dealing with text problems, one of the first things you have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of features vector for each token (the size of the feature vector can be defined and this embedding can be learned).

### Text vectorization(tokenization)


In [18]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [20]:
#using the default TextVectorization parameters
text_vectorizer = TextVectorization(
    max_tokens=None, #how many words in vocabulary (automatically add <OOV> for out of vocabulary)
    standardize = "lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None, #create groups of n words
    output_mode = "int", #how to map tokens into numbers
    output_sequence_length = None, #how long do you want your sequences to be
    pad_to_max_tokens = False
)

2024-04-23 03:42:55.171853: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-23 03:42:55.171882: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-23 03:42:55.171889: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-23 03:42:55.171913: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-23 03:42:55.171935: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [21]:
# find the average number of tokens (words) in training tweets.
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [22]:
#setup text vectorization variables
max_vocab_length = 10000 #max number of words in the vocabulary
max_length = 15 #max length of our sequences will be (e.g. how many words from a tweet does a model see)

text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True
)

In [23]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [24]:
# Create a sample sentence and tokenize it
sample_sentence = "There is a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
#choose a random sentence from the training data and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence}\n vercotized version:{text_vectorizer([random_sentence])}")

Original text : 
 Ignition Knock (Detonation) Sensor-KNOCK SENSOR Delphi AS10004 http://t.co/LMrKgPOrcF http://t.co/6WAdNmsTOv
 vercotized version:[[ 888  885  580 8657 1767    1    1    1    1    0    0    0    0    0
     0]]


In [26]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() #Get all the unique words in the train data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words in the vocab
print(f"Total number of words in the vocab : {len(words_in_vocab)}")
print(f"5  most common words in the vocab : {top_5_words}")
print(f"5 least common words in vocab : {bottom_5_words}")

Total number of words in the vocab : 10000
5  most common words in the vocab : ['', '[UNK]', 'the', 'a', 'in']
5 least common words in vocab : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

To make our embedding, we are going to use TensorFlow's embedding layer.

The parameters we care most about for the embedding layer
* `input_dim` = size of the vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long.
* `input_length` = length of the sequence being passed to the embedding layer.


In [27]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim = max_vocab_length, #set input shape
    output_dim = 128, #output shape commonly used multples of 8, can choose any value
    input_shape = (max_length,), #how long is each input
)
embedding

/opt/anaconda3/envs/deeplearning/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


<Embedding name=embedding, built=False>

In [28]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence}\n Embedded version:")

# Embedd the random sentences (turn it into dense vectors of fixed size)
sample_embed = text_vectorizer([sample_sentence])
sample_embed = embedding(sample_embed)
sample_embed

Original text : 
 @JWalkerLyle Discovered by @NickCannon 
 Listen/Buy @realmandyrain #RIOT on @iTunesMusic @iTunes https://t.co/dehMym5lpk Û_ #BlowMandyUp
 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00364857, -0.02656436,  0.01976028, ..., -0.00349844,
         -0.04563659,  0.02616617],
        [-0.01369131,  0.01943295, -0.04640696, ...,  0.04050651,
          0.00691215, -0.01371105],
        [ 0.03127196,  0.03207911,  0.02820811, ..., -0.04005416,
         -0.04435724, -0.01227663],
        ...,
        [-0.02424335,  0.02293129,  0.02061732, ...,  0.00914634,
         -0.01805437,  0.04164337],
        [-0.02424335,  0.02293129,  0.02061732, ...,  0.00914634,
         -0.01805437,  0.04164337],
        [-0.02424335,  0.02293129,  0.02061732, ...,  0.00914634,
         -0.01805437,  0.04164337]]], dtype=float32)>

In [29]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00364857, -0.02656436,  0.01976028,  0.02622975, -0.0493608 ,
         0.01165793, -0.03349875,  0.02456752, -0.04028182,  0.00022739,
        -0.01001573,  0.03072914,  0.01929524,  0.04945544,  0.00849485,
        -0.01597029, -0.01004722,  0.02689492,  0.02172258,  0.03362665,
        -0.01686987,  0.01024245,  0.04336845, -0.0313458 , -0.00514204,
        -0.03632387, -0.02699281,  0.02188778,  0.04443175,  0.01580569,
        -0.01114417, -0.03727986,  0.03636273,  0.01743204,  0.03716694,
         0.02631445, -0.02621396,  0.02189006,  0.04974555,  0.04211069,
        -0.01078826,  0.04124681, -0.04916097, -0.03348986,  0.03304471,
         0.00390191,  0.04925014,  0.01811733,  0.01508491, -0.02725167,
        -0.01338415,  0.04520184, -0.01211004,  0.03395183,  0.02013055,
        -0.0001106 ,  0.01119969, -0.0142042 ,  0.00249925, -0.04433658,
        -0.04706002, -0.01530577, -0.0298327 ,  0.02477256,  0.03712403,
  

## Modelling a text dataset (running a series of experiments)

Now we have a way to turn our text sequence into numbers, it' time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-Forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bi directional-LSTM model(RNN)
* Model 5: 1D convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub pretrained feature extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% data

We will follow the standard steps in modelling with TensorFlow
* Create the model
* Build the model
* Fit the model
* Evaluate the model



### Model 0: Getting a baseline

As with all machine learning modelling experiments, it is important to create a baseline model so you have a bench mark for future experiments to build a model.

To create our baseline we'll use sklearns multi nomial naive bayes using TF-IDF formula to convert our words to numbers

> **NOTE:** It is common to use non-Deep learning algorithmns as a baseline due to their speed. Later using deep learning algorihmns to see if you can improve upon them.

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers using tfidf
    ("clf", MultinomialNB()) #model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model acheives an accuracy of : {baseline_score * 100:.2f}%")

Our baseline model acheives an accuracy of : 79.27%


In [32]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating evaluation functions for our model experiments

We can evaluate all of our model's predictions with everytime, but this will be cumbersome. So, let's create afunction to calculate the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Creating an evaluation function to evaluate our model
def calculate_results(y_true, y_preds):
    """
    Accepts the true labels and pred labels as input and return a dictionary
    of Accuracy, Precison, Recall, F1-Score for binary classification.
    
    args:
    y_true(np.array): truth labels 
    y_preds(np.array): prediction labels

    returns:
    metrics(dict): a dictionary
    of Accuracy, Precison, Recall, F1-Score
    """
    metrics = dict()
    metrics["accuracy"] = accuracy_score(y_true, y_preds)
    metrics["precision"] = precision_score(y_true, y_preds, average="weighted")
    metrics["recall"] = recall_score(y_true, y_preds, average="weighted")
    metrics["f1-score"] = f1_score(y_true, y_preds, average="weighted")
    print(f"Accuracy is : {metrics['accuracy'] * 100:.2f}%")
    print(f"Precision Score is : {metrics['precision']:.2f}")
    print(f"Recall score is : {metrics['recall']:.2f}")
    print(f"F1-Score is : {metrics['f1-score']:.2f}")
    return metrics

In [34]:
baselineline_resutls =calculate_results(val_labels, baseline_preds)

Accuracy is : 79.27%
Precision Score is : 0.81
Recall score is : 0.79
F1-Score is : 0.79


### Model 1: Feed Forward Neural Network Model



In [35]:
# Create a tensorboard callback need to create a new one for each model

from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "NLP/model_logs"



In [36]:
# Build model with functional API

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimentional

x = text_vectorizer(inputs) # turn the input text into numbers

x = embedding(x) #create an embedding of the numbers
x = layers.GlobalAveragePooling1D(name="global_avg_pooling_1D")(x) # to fix the dimension incompatiablity issue
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer for binary classification
model_1 = tf.keras.Model(inputs, outputs,name="model_1_dense")


In [37]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pooling_1D           │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
# compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer = "adam",
                metrics=["accuracy"])

In [39]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_1_dense")])


Saving TensorBoard log files to: NLP/model_logs/model_1_dense/20240423-034255
Epoch 1/5


2024-04-23 03:42:56.441417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6447 - loss: 0.6469 - val_accuracy: 0.7677 - val_loss: 0.5302
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8144 - loss: 0.4568 - val_accuracy: 0.7822 - val_loss: 0.4799
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8665 - loss: 0.3496 - val_accuracy: 0.7861 - val_loss: 0.4584
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8950 - loss: 0.2853 - val_accuracy: 0.7887 - val_loss: 0.4639
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9172 - loss: 0.2351 - val_accuracy: 0.7874 - val_loss: 0.4804


In [40]:
#check the results

model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7817 - loss: 0.5140


[0.48044681549072266, 0.787401556968689]

In [41]:
# Make some predictions and evaluate them
model_1_pred_probs = model_1.predict(val_sentences)
#convert model predictions to label format
model_1_preds = tf.round(model_1_pred_probs)
#calculate the model results
model_1_results =calculate_results(val_labels, model_1_preds)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Accuracy is : 78.74%
Precision Score is : 0.79
Recall score is : 0.79
F1-Score is : 0.78


In [42]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baselineline_resutls.values()))

array([False, False, False, False])

## Visualize learned embeddings



In [43]:
# Get the vocabulary from the text vectoization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
#model 1 summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pooling_1D           │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [45]:
# Get the weight matrix of the embedding layer

# these are the numerical representaion of each token in our training data, which have been learned for 5 epochs

embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights 

[array([[-0.03384594,  0.00865505,  0.00322512, ..., -0.00060821,
         -0.00314511,  0.05916424],
        [ 0.03628474,  0.00402269,  0.01217359, ..., -0.03746361,
         -0.01951195,  0.02728236],
        [-0.05949177,  0.03521245, -0.00058741, ...,  0.02855311,
          0.02599362,  0.0630648 ],
        ...,
        [ 0.00878338,  0.00384588,  0.03092171, ..., -0.03115517,
          0.0433116 , -0.01919413],
        [-0.05739041, -0.0801575 , -0.05887246, ..., -0.01887629,
          0.01918228,  0.02917477],
        [-0.07052994, -0.11291071, -0.0185412 , ..., -0.10057747,
          0.0334874 ,  0.04546812]], dtype=float32)]

In [46]:
embed_weights[0].shape #same size as vocab size and embeding_dim (output of the embedding layer)

(10000, 128)

Now we have the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so TensorFlow has a handy tool called projector:https://projector.tensorflow.org/

Check out the guide for word embeddings : 
https://www.tensorflow.org/text/guide/word_embeddings


In [47]:
# create embedding files(we got this from TensorFlow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [48]:
# !mkdir nlp_vecor_visualization #uncomment to create folder again

!mv vectors.tsv nlp_vecor_visualization
!mv metadata.tsv nlp_vecor_visualization

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of the previous input to aid the representation of a later input.

If you want the overview of the internals of the Recurrent neural network, see the following:
- MIT's sequence modelling lecture 
- Andrej Karpathy's the unreasonable effectiveness of Recurrent neural network - https://karpathy.github.io/2015/05/21/rnn-effectiveness/



### Model 2: LSTM

LSTM = Long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like
```
Input(text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output(label probability)
```


In [49]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) #when you are stacking RNN cells together you need to return_sequences = True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [50]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [52]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(train_sentences,train_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: NLP/model_logs/model_2_LSTM/20240423-034319
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.9045 - loss: 0.2864 - val_accuracy: 0.9435 - val_loss: 0.1585
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9435 - loss: 0.1581 - val_accuracy: 0.9583 - val_loss: 0.1169
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9536 - loss: 0.1262 - val_accuracy: 0.9641 - val_loss: 0.1009
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9620 - loss: 0.1003 - val_accuracy: 0.9729 - val_loss: 0.0841
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9701 - loss: 0.0781 - val_accuracy: 0.9759 - val_loss: 0.0725


In [53]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


array([[0.15283544],
       [0.78276443],
       [0.99963605],
       [0.09694918],
       [0.00171379],
       [0.998553  ],
       [0.70146215],
       [0.9997743 ],
       [0.99966383],
       [0.39916173]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels
model_2_preds = tf.round(model_2_pred_probs)
model_2_preds[:10]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]], dtype=float32)>

In [55]:
# Calculate the model  results
model_2_results = calculate_results(val_labels, model_2_preds)

Accuracy is : 77.17%
Precision Score is : 0.77
Recall score is : 0.77
F1-Score is : 0.77


In [56]:
baselineline_resutls

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

### Model 3 : GRU(Gated Recurrent Units)

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parametes.

In [57]:
# Let's recrete the model 2 with GRU insted of LSTM

inputs = layers.Input(shape=(1,), dtype="string")
#passing the inputs to text vectorization layer
x = text_vectorizer(inputs)
#embedding 
x = embedding(x)
#creating a GRU with return sequences as False
# x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu") (x)
#creating output layer
outputs = layers.Dense(1, activation="sigmoid")(x)

#creating model_3 from inputs and outputs
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [58]:
# compiling the model
model_3.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [59]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [60]:
# fitting the model for 5 epoch
history_model_3 = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"GRU_Model")])

Saving TensorBoard log files to: NLP/model_logs/GRU_Model/20240423-034352


Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.8975 - loss: 0.2595 - val_accuracy: 0.7743 - val_loss: 0.8830
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9705 - loss: 0.0828 - val_accuracy: 0.7717 - val_loss: 0.8161
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9771 - loss: 0.0614 - val_accuracy: 0.7782 - val_loss: 0.8426
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9788 - loss: 0.0565 - val_accuracy: 0.7782 - val_loss: 1.1910
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9790 - loss: 0.0532 - val_accuracy: 0.7743 - val_loss: 1.2649


In [61]:
model_3.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7523 - loss: 1.4127


[1.2648653984069824, 0.7742782235145569]

In [62]:
# Make some predicitons with our model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


array([[6.3037145e-04],
       [8.4304392e-01],
       [9.9986255e-01],
       [1.2428607e-02],
       [7.1711307e-05],
       [9.9954236e-01],
       [8.9079863e-01],
       [9.9992645e-01],
       [9.9986744e-01],
       [2.3819923e-01]], dtype=float32)

In [63]:
# Convert the model_3 pred probs to labels
model_3_preds = tf.round(model_3_pred_probs)
model_3_preds[:5]

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [1.],
       [0.],
       [0.]], dtype=float32)>

In [64]:
# Calculate model 3 results
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

Accuracy is : 77.43%
Precision Score is : 0.78
Recall score is : 0.77
F1-Score is : 0.77


{'accuracy': 0.7742782152230971,
 'precision': 0.7762579677540307,
 'recall': 0.7742782152230971,
 'f1-score': 0.7721023076072874}

### Model 4 : Bidirectional RNN

Normal RNN's go from left to right (just like you read an English sentence) however, a biderectional RNN goes from right to left and left to right.

In [82]:
#creating input layer
inputs = layers.Input(shape=(1,), dtype="string")
#pasing inputs to text_vectorizer
x = text_vectorizer(inputs)
#passing the inputs to embedding layer
x = embedding(x)
# creting bidirectional layer
# x = layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")


In [83]:
#compiling the model
model_4.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [84]:
# fitting the model for 5 epochs
history_model_4 = model_4.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data = (val_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_Bidirectional_LSTM")])

Saving TensorBoard log files to: NLP/model_logs/model_4_Bidirectional_LSTM/20240423-035901
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.9376 - loss: 0.1720 - val_accuracy: 0.7572 - val_loss: 1.0826
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9815 - loss: 0.0375 - val_accuracy: 0.7546 - val_loss: 1.3034
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9825 - loss: 0.0362 - val_accuracy: 0.7625 - val_loss: 1.3662
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9821 - loss: 0.0344 - val_accuracy: 0.7664 - val_loss: 1.4130
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9776 - loss: 0.0565 - val_accuracy: 0.7677 - val_loss: 1.3633


Adding bidirectionality in models increse the training time

In [78]:
model_4.summary()

Model: "model_4_Bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 15, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,360,325 (16.63 MB)

 Trainable params: 1,453,441 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,906,884 (11.09 MB)

In [85]:
# Making predictions with our model_4
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


array([[3.8891381e-03],
       [8.5180813e-01],
       [9.9973398e-01],
       [1.8771821e-01],
       [2.7620694e-05]], dtype=float32)

In [86]:
#rounding the pred probs
model_4_preds = tf.round(model_4_pred_probs)
model_4_preds[:5]

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [1.],
       [0.],
       [0.]], dtype=float32)>

In [87]:
#evaluating the model 
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

Accuracy is : 76.77%
Precision Score is : 0.77
Recall score is : 0.77
F1-Score is : 0.77


{'accuracy': 0.7677165354330708,
 'precision': 0.7686883789989795,
 'recall': 0.7677165354330708,
 'f1-score': 0.7659262393850468}

In [88]:
model_3_results

{'accuracy': 0.7742782152230971,
 'precision': 0.7762579677540307,
 'recall': 0.7742782152230971,
 'f1-score': 0.7721023076072874}

Our Bidirectional model performed worse than the baseline model.

## Convolutinal Neural Networks for Text (and other type of sequences)

We have used CNN's for images, but images are typically 2D(height and width).. however, our text data is 1D

Previously we have used Conv2D for our image data but now we are going to use Conv1D

The typical structure of a Conv1D model for sequences:

```
Inputs(text) -> Tokenization -> Embedding -> Layer(s)(typically Conv1D + pooling) -> Outputs(class probabilities)

### Model 5:Conv1D

In [102]:
#creating a model with 1D conv layer

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64, 3)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid") (x)

model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1D")

In [100]:
#compiling the model
model_5.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [101]:
#fit the model
model_5.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.9545 - loss: 0.1313 - val_accuracy: 0.7493 - val_loss: 1.1242
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9834 - loss: 0.0436 - val_accuracy: 0.7493 - val_loss: 1.2860
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9784 - loss: 0.0513 - val_accuracy: 0.7480 - val_loss: 1.4641
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9788 - loss: 0.0494 - val_accuracy: 0.7585 - val_loss: 1.3431
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9847 - loss: 0.0361 - val_accuracy: 0.7559 - val_loss: 1.3105
